In [1]:
import numpy as np

# charge le fichier (ignore les lignes qui commencent par #)
d = np.loadtxt("ddscat7.3.4_250505/ddpostprocess_E3D.out", comments="#")

# reconstruire Ex, Ey, Ez complexes
Ex = d[:, 0] + 1j * d[:, 1]
Ey = d[:, 2] + 1j * d[:, 3]
Ez = d[:, 4] + 1j * d[:, 5]

# |E|^2
Esq = np.abs(Ex)**2 + np.abs(Ey)**2 + np.abs(Ez)**2

print(Esq[:10])  # affiche les 10 premiers points


[0.62423211 0.69262831 0.60347365 0.64117423 0.69057887 0.72277678
 0.71979682 0.62041575 0.60903646 0.71422724]


In [3]:
d = np.loadtxt("adda/src/seq/run034_sphere_g15_m1.302/IntField-Y", skiprows=1)
Esq_adda = d[:, 3]  # déjà en |E|^2

print(Esq_adda[:10])  # affiche les 10 premiers points

[0.63706368 0.59304064 0.63706368 0.61255346 0.70797927 0.71517229
 0.70797927 0.61255346 0.71378437 0.72753615]


In [5]:
print(Esq.sum() - Esq_adda.sum())

-5.793532438757666


## Polarization comparison

In [229]:
import numpy as np

d = np.loadtxt("ddscat7.3.4_250505/ddpostprocess_P3D.out", comments="#")

Px = d[:, 3] + 1j * d[:, 4]
Py = d[:, 5] + 1j * d[:, 6]
Pz = d[:, 7] + 1j * d[:, 8]

# |P|^2
Psq = np.abs(Px)**2 + np.abs(Py)**2 + np.abs(Pz)**2

print(Psq[:10])

[10542.79937312 10775.43048751 11077.75022266  9933.43045776
 11368.11135808 11509.09285828 11904.36402071 10906.91053508
 10530.64220912 12191.52654685]


In [230]:
d[:,0].max()

np.float64(92.91711449671838)

In [231]:
d = np.loadtxt("adda/src/seq/run006_sphere_g15_m1.302/DipPol-Y", skiprows=1)
Psq_adda = d[:, 3]  # déjà en |P|^2

print(Psq_adda[:10])

[10542.79937312 10775.43048751 11077.75022266  9933.43045776
 11368.11135808 11509.09285828 11904.36402071 10906.91053508
 10530.64220912 12191.52654685]


In [232]:
print(Psq.sum() - Psq_adda.sum())

0.0


In [233]:
print(Psq.min(), Psq.max(), Psq.mean(), Psq.sum(), np.median(Psq), Psq.std())
print(Psq_adda.min(), Psq_adda.max(), Psq_adda.mean(), Psq_adda.sum(), np.median(Psq_adda), Psq_adda.std())

9920.604538523212 18666.197791172508 14471.046115846579 25917643.593481224 14180.503934438952 1952.9133952048692
9920.604538523208 18666.197791172526 14471.046115846579 25917643.593481224 14180.503934438928 1952.9133952048703


In [234]:
abs((Psq.sum()-Psq_adda.sum())/Psq_adda.sum())

np.float64(0.0)

In [222]:
import math

int(-math.log10(abs((Psq.sum()-Psq_adda.sum())/Psq_adda.sum())))

15

In [223]:
print(np.sort(Psq)[:20])

[9.92060454e-15 9.92060454e-15 9.92060454e-15 9.92060454e-15
 9.93343046e-15 9.93343046e-15 9.93343046e-15 9.93343046e-15
 1.03303346e-14 1.03303346e-14 1.03303346e-14 1.03303346e-14
 1.05306422e-14 1.05306422e-14 1.05427994e-14 1.05427994e-14
 1.05427994e-14 1.05427994e-14 1.07754305e-14 1.07754305e-14]


In [224]:
print(np.sort(Psq_adda)[:20])

[9.92060454e-15 9.92060454e-15 9.92060454e-15 9.92060454e-15
 9.93343046e-15 9.93343046e-15 9.93343046e-15 9.93343046e-15
 1.03303346e-14 1.03303346e-14 1.03303346e-14 1.03303346e-14
 1.05306422e-14 1.05306422e-14 1.05427994e-14 1.05427994e-14
 1.05427994e-14 1.05427994e-14 1.07754305e-14 1.07754305e-14]


In [225]:
print(np.sort(Psq)[-20:])

[1.83563854e-14 1.83580415e-14 1.83580415e-14 1.83713416e-14
 1.83892389e-14 1.83892389e-14 1.84253794e-14 1.84317641e-14
 1.84317641e-14 1.84370602e-14 1.84370602e-14 1.85379455e-14
 1.85379455e-14 1.85379455e-14 1.85379455e-14 1.85996121e-14
 1.85996121e-14 1.86428525e-14 1.86428525e-14 1.86661978e-14]


In [226]:
print(np.sort(Psq_adda)[-20:])

[1.83563854e-14 1.83580415e-14 1.83580415e-14 1.83713416e-14
 1.83892389e-14 1.83892389e-14 1.84253794e-14 1.84317641e-14
 1.84317641e-14 1.84370602e-14 1.84370602e-14 1.85379455e-14
 1.85379455e-14 1.85379455e-14 1.85379455e-14 1.85996121e-14
 1.85996121e-14 1.86428525e-14 1.86428525e-14 1.86661978e-14]


In [227]:
print(np.sort(Psq)[20:]-np.sort(Psq_adda)[20:])

[ 2.20881053e-29  3.15544362e-29 -4.73316543e-30 ... -3.78653235e-29
 -2.52435490e-29 -4.41762107e-29]


In [228]:
print(Psq.sum() - Psq_adda.sum())

3.2311742677852644e-27
